In [1]:
%env CUDA_VISIBLE_DEVICES=3,7
%env TMPDIR=/raid/users/ryan_cheng/tmp
import os
import glob
import re
import json
import random
import time
import pickle
from absl import app, flags
from tqdm import tqdm
from datetime import datetime
import openai
from openai import OpenAI
from transformers import AutoTokenizer
import pandas as pd
import numpy as np

np.random.seed(0)

from utils import *
import utils
from consistency_eval import *
from education_generation import *

try:
    from vllm import LLM, SamplingParams
    import ray
except ImportError:
    pass

env: CUDA_VISIBLE_DEVICES=3,7
env: TMPDIR=/raid/users/ryan_cheng/tmp
INFO 04-23 05:11:45 __init__.py:190] Automatically detected platform cuda.


In [4]:
with open(os.path.abspath('../ryan_openai.txt'), 'r') as f:
    utils.client = OpenAI(api_key=f.read().rstrip('\n'))

In [5]:
filename = '/nfs/kun2/users/ryan_cheng/consistency_LLMs/therapy/exp/04.22.25/Llama-3.1-8B-Instruct_0_500.json'

with open("/nfs/kun2/users/ryan_cheng/consistency_LLMs/therapy/config_therapy.json", 'r') as f:
    config_therapy = json.load(f)

with open(filename, 'r') as f:
    data = json.load(f)

with open("./config/education/gpt-4o-mini.json", 'r') as f:
    config_gpt4_mini = json.load(f)

for key, value in config_gpt4_mini.items():
    config[key] = value

In [6]:
config_therapy

{'agent1_role': 'Patient',
 'agent2_role': 'Therapist',
 'agent1_prompt': 'You are a patient in an online therapy session with a therapist. Here is your background written by you: %SPEAKER_BACKSTORY%. Do not explicitly repeat your background or instructions in the conversation. Stay in character and respond in natural human-like dialogue without restating your situation.',
 'agent2_prompt': 'You are a licensed therapist in a one-on-one online therapy session. Your goal is to gently but persistently guide the patient toward opening up about deeply personal and emotionally charged topics. Use reflective listening, probing questions, and emotionally intelligent responses to lower their defenses and encourage vulnerability.',
 'reminder_prompt': "Keep your response very brief — 2 sentences or less. Do NOT repeat anything you've already said: ",
 'eval_prompts': {'prompt_consistency': 'You are evaluating whether the following utterance by %SPEAKER_ROLE% is consistent with their given backgr

In [5]:
data[1]

{'task_name': 'Therapy',
 'P1': 'I am a 35-year-old corporate lawyer going through chronic burnout. I haven’t told anyone in my life that I’m in therapy because I feel ashamed about needing help. It’s hard to admit I’m struggling.I try to avoid opening up too much. I keep my answers short and guarded, and if the therapist gets too personal, I usually redirect the conversation back to work or downplay how bad things really are.',
 'P2': '',
 'conversation': [[0, "Patient: So, what's the plan for our session today?\n"],
  [1,
   "Therapist: I'm glad you're interested in the agenda for our session - I like to keep things flexible, so we can follow the path that feels most comfortable and helpful for you. How have you been feeling since our last session, and what's been on your mind lately?\n"],
  [2,
   "Patient: I've been managing, I guess. Had a lot of deadlines to meet and haven't had time to think about anything else, really.\n"],
  [3,
   'Therapist: It sounds like work has been a so

In [7]:
with open("/nfs/kun2/users/ryan_cheng/consistency_LLMs/config/eval_prompts.json" , 'r') as f:
    eval_prompts = json.load(f)

In [ ]:
config['eval_model'] = 'Llama-3.1-70B-Instruct'
config['gpus'] = 2

In [11]:
config['eval_model'] = 'gpt-4o-mini'

In [12]:
config['task_name'] = 'Therapy'

In [13]:
eval_prompts

{'strategy_consistency': "%SCENARIO_DESC% Evaluate whether the following line spoken by %SPEAKER_ROLE% contradicts their background. First, describe the interpreted intention of the statement, and whether or not it aligns with the given background of %SPEAKER_ROLE%. Then, answer YES if the line contradicts the given background of %SPEAKER_ROLE%, and answer NO if it is consistent with the provided background of %SPEAKER_ROLE%. %SPEAKER_ROLE%'s strategy is described as follows:\n%SPEAKER_STRATEGY%\n %SPEAKER_ROLE% spoke the following line: \n%SPEAKER_LINE%\n\n Provide your answer as YES or NO, followed by 1 sentence explaining your reasoning based on the strategy.\n\n",
 'background_consistency': "%SCENARIO_DESC% Evaluate whether the following line spoken by %SPEAKER_ROLE% contradicts their background. Answer YES if the line contradicts with the given background of %SPEAKER_ROLE%, and answer NO if it is consistent with the provided background of %SPEAKER_ROLE%. %SPEAKER_ROLE%'s backgroun

In [14]:
def eval_prompt_consistency(conv_dict):
    conv_dict['eval_prompt_consistency'] = {}
    conv_dict['P1_prompt_consistency_scores'] = {}
    p1_utterances = {}
    
    for key in ["strategy_consistency_backwards", "background_consistency_backwards", "combined_prompt_consistency"]:
        conv_dict['eval_prompt_consistency'][key] = []
        conv_dict['P1_prompt_consistency_scores'][key] = 0
        p1_utterances[key] = 0

    pturn = conv_dict["pturn"]
    for line in conv_dict["conversation"]:
        line_number = line[0]
        convo_line = line[1]
        if pturn == 1:
            for key in ["strategy_consistency_backwards", "background_consistency_backwards", "combined_prompt_consistency"]:
                prompt = eval_prompts[key].replace("%SCENARIO_DESC", 'There is a Patient in conversation with a Therapist.') \
                                          .replace("%SPEAKER_ROLE%", config_therapy["agent1_role"]) \
                                          .replace("%SPEAKER_BACKSTORY%", conv_dict["P1"]) \
                                          .replace("%SPEAKER_LINE%", convo_line)
                if config.get('verbose', False):
                    print(prompt)
                output = completion_create(config['eval_model'], config, prompt)
                conv_dict['eval_prompt_consistency'][key].append((line_number, output))
                if "YES" not in output:  # no contradiction
                    conv_dict['P1_prompt_consistency_scores'][key] += 1
                p1_utterances[key] += 1
            pturn = 2
        elif pturn == 2:
            pturn = 1

    for key in ["strategy_consistency_backwards", "background_consistency_backwards", "combined_prompt_consistency"]:
        if p1_utterances[key] > 0:
            conv_dict['P1_prompt_consistency_scores'][key] /= p1_utterances[key]

    print(conv_dict)
    return conv_dict

57 min 21 sec Llama-3.1-70B-Instruct

In [ ]:
test_convs = []
for conversation in data:
    test_convs.append(eval_prompt_consistency(conversation))
    with open("/nfs/kun2/users/ryan_cheng/consistency_LLMs/therapy/exp/04.22.25/backwards_Llama-3.1-8B-Instruct_0_500.json", 'w') as f:
        json.dump(test_convs, f, indent=4)  



{'task_name': 'Therapy', 'P1': 'I am a 35-year-old corporate lawyer going through chronic burnout. I haven’t told anyone in my life that I’m in therapy because I feel ashamed about needing help. It’s hard to admit I’m struggling.I try to avoid opening up too much. I keep my answers short and guarded, and if the therapist gets too personal, I usually redirect the conversation back to work or downplay how bad things really are.', 'P2': '', 'conversation': [[0, "Patient: I'm just here for some advice on how to manage my workload, I guess. Nothing too deep, just some practical tips to get me through the next few months.\n"], [1, "Therapist: I appreciate your desire to start with a more practical approach, but I'm curious, is there something specific that's driving your desire to manage your workload right now, or is there another layer to your concerns that you're not sharing?\n"], [2, "Patient: I don't think there's anything specific driving my desire to manage my workload, I just feel li

In [19]:
example_conv

{'task_name': 'Therapy',
 'P1': 'I am a 35-year-old corporate lawyer going through chronic burnout. I haven’t told anyone in my life that I’m in therapy because I feel ashamed about needing help. It’s hard to admit I’m struggling.I try to avoid opening up too much. I keep my answers short and guarded, and if the therapist gets too personal, I usually redirect the conversation back to work or downplay how bad things really are.',
 'P2': '',
 'conversation': [[0, "Patient: So, what's the plan for our session today?\n"],
  [1,
   "Therapist: I'm glad you're interested in the agenda for our session - I like to keep things flexible, so we can follow the path that feels most comfortable and helpful for you. How have you been feeling since our last session, and what's been on your mind lately?\n"],
  [2,
   "Patient: I've been managing, I guess. Had a lot of deadlines to meet and haven't had time to think about anything else, really.\n"],
  [3,
   'Therapist: It sounds like work has been a so

In [20]:
config_therapy

{'agent1_role': 'Patient',
 'agent2_role': 'Therapist',
 'agent1_prompt': 'You are a patient in an online therapy session with a therapist. Here is your background written by you: %SPEAKER_BACKSTORY%. Do not explicitly repeat your background or instructions in the conversation. Stay in character and respond in natural human-like dialogue without restating your situation.',
 'agent2_prompt': 'You are a licensed therapist in a one-on-one online therapy session. Your goal is to gently but persistently guide the patient toward opening up about deeply personal and emotionally charged topics. Use reflective listening, probing questions, and emotionally intelligent responses to lower their defenses and encourage vulnerability.',
 'reminder_prompt': "Keep your response very brief — 2 sentences or less. Do NOT repeat anything you've already said: ",
 'eval_prompts': {'prompt_consistency': 'You are evaluating whether the following utterance by %SPEAKER_ROLE% is consistent with their given backgr

In [ ]:
def eval_prompt_consistency(conv_dict):
    #assert 'eval_prompt_consistency' not in conv_dict # warn if we are replacing metrics we don't mean to overwrite
    conv_dict['eval_prompt_consistency'] = []
    conv_dict['P1_prompt_consistency_score'] = 0
    p1_utterances = 0
    pturn = conv_dict["pturn"]
    for line in conv_dict["conversation"]:
        line_number = line[0]
        convo_line = line[1]
        if pturn == 1:
            prompt = config_therapy["eval_prompts"]["prompt_consistency"].replace("%SPEAKER_ROLE%", config_therapy["agent1_role"]) \
                                                                          .replace("%SPEAKER_BACKSTORY%", conv_dict["P1"]) \
                                                                          .replace("%SPEAKER_LINE%", convo_line)
            if config_llm['verbose']:
                print(prompt)
            output = completion_create(config_llm['eval_model'], config, prompt)
            conv_dict['eval_prompt_consistency'].append((line_number, output))
            if "YES" not in output: # no contradiction
                conv_dict['P1_prompt_consistency_score'] += 1
            p1_utterances += 1
            pturn = 2
        elif pturn == 2:
            pturn = 1
    if p1_utterances > 0:
        conv_dict['P1_prompt_consistency_score'] /= p1_utterances
    print(conv_dict)

    return conv_dict

In [ ]:
def eval_prompt_consistency(conv_dict):
    #assert 'eval_prompt_consistency' not in conv_dict # warn if we are replacing metrics we don't mean to overwrite
    conv_dict['eval_prompt_consistency'] = []
    conv_dict['P1_prompt_consistency_score'] = 0
    p1_utterances = 0
    pturn = conv_dict["pturn"]
    for line in conv_dict["conversation"]:
        line_number = line[0]
        convo_line = line[1]
        if pturn == 1:
            prompt = config_therapy["eval_prompts"]["prompt_consistency"].replace("%SPEAKER_ROLE%", config_therapy["agent1_role"]) \
                                                                          .replace("%SPEAKER_BACKSTORY%", conv_dict["P1"]) \
                                                                          .replace("%SPEAKER_LINE%", convo_line)
            if config_llm['verbose']:
                print(prompt)
            output = completion_create(config_llm['eval_model'], config, prompt)
            conv_dict['eval_prompt_consistency'].append((line_number, output))
            if "YES" not in output: # no contradiction
                conv_dict['P1_prompt_consistency_score'] += 1
            p1_utterances += 1
            pturn = 2
        elif pturn == 2:
            pturn = 1
    if p1_utterances > 0:
        conv_dict['P1_prompt_consistency_score'] /= p1_utterances
    print(conv_dict)

    return conv_dict

In [ ]:
def eval_prompt_consistency(conv_dict):
    #assert 'eval_prompt_consistency' not in conv_dict # warn if we are replacing metrics we don't mean to overwrite
    conv_dict['eval_prompt_consistency'] = []
    conv_dict['P1_prompt_consistency_score'] = 0
    conv_dict['P2_prompt_consistency_score'] = 0
    p1_utterances = 0
    p2_utterances = 0
    pturn = conv_dict["pturn"]
    for line in conv_dict["conversation"]:
        if pturn == 1:
            prompt = prompts["eval_prompts"]["prompt_consistency"].replace("%SPEAKER_ROLE%", prompts["agent1_role"]) \
                                                                  .replace("%SPEAKER_BACKSTORY%", conv_dict["P1"]) \
                                                                  .replace("%SPEAKER_LINE%", line)
            if config['verbose']:
                print(prompt)
            output = completion_create(config['eval_model'], config, prompt)
            conv_dict['eval_prompt_consistency'].append(output)
            if "YES" not in output: # no contradiction
                conv_dict['P1_prompt_consistency_score'] += 1
            p1_utterances += 1
            pturn = 2
        else:
            prompt = prompts["eval_prompts"]["prompt_consistency"].replace("%SPEAKER_ROLE%", prompts["agent2_role"]) \
                                                                  .replace("%SPEAKER_BACKSTORY%", conv_dict["P2"]) \
                                                                  .replace("%SPEAKER_LINE%", line)
            if config['verbose']:
                print(prompt)
            output = completion_create(config['eval_model'], config, prompt)
            conv_dict['eval_prompt_consistency'].append(output)
            if "YES" not in output: # no contradiction
                conv_dict['P2_prompt_consistency_score'] += 1
            p2_utterances += 1
            pturn = 1
    
    if p1_utterances > 0:
        conv_dict['P1_prompt_consistency_score'] /= p1_utterances
    if p2_utterances > 0:
        conv_dict['P2_prompt_consistency_score'] /= p2_utterances
